In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json
import os
import pandas as pd
pd.set_option("display.max_columns" , 30)
import requests

In [3]:
"""
1.get the data from s3
2.weather data transformations
3.taxi_trips transformations
4.update payment_type_master
5.update company_master
6.updatte taxi_trips with company and payment_type ids (replace the string values with ids from the latest amster table)
7.upload weather data to s3
8.upload taxi data to s3
9.upload the newest payment_type_master and company _master





"""

'\n1.get the data from s3\n2.weather data transformations\n3.taxi_trips transformations\n4.update payment_type_master\n5.update company_master\n6.updatte taxi_trips with company and payment_type ids (replace the string values with ids from the latest amster table)\n7.upload weather data to s3\n8.upload taxi data to s3\n9.upload the newest payment_type_master and company _master\n\n\n\n\n\n'

#### taxi_trips transformation codes

In [3]:
current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime= current_datetime.strftime("%Y-%m-%d")

url= f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' AND trip_start_timestamp <='{formatted_datetime}T23:59:59'&$limit=30000"

headers= {"X-App-Token" : os.environ.get("CHICAGO_API_TOKEN")}
response= requests.get(url)
data= response.json()

In [4]:
taxi_trips =  pd.DataFrame(data)
#taxi_trips.head()

In [5]:
taxi_trips.drop(["pickup_census_tract","dropoff_census_tract"], axis=1, inplace=True)
taxi_trips.drop(["pickup_centroid_location","dropoff_centroid_location"], axis=1, inplace=True)

taxi_trips.dropna(inplace=True)

taxi_trips.rename(columns={"pickup_community_area":"pickup_community_area_id",
                  "dropoff_community_area":"dropoff_community_area_id"}, inplace=True)

taxi_trips["datetime_for_weather"]= pd.to_datetime(taxi_trips["trip_start_timestamp"])
taxi_trips["datetime_for_weather"]= taxi_trips["datetime_for_weather"].dt.floor("h")

In [7]:
taxi_trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tips,tolls,extras,trip_total,payment_type,company,dropoff_community_area_id,dropoff_centroid_latitude,dropoff_centroid_longitude,pickup_community_area_id,pickup_centroid_latitude,pickup_centroid_longitude,datetime_for_weather
2,fc617008a8964db298cb7d62d7c09a7331349d0b,6201cd8256e26b0ef547ca53db3aaac4d23fc7d02e4c9f...,2024-10-11T23:45:00.000,2024-10-11T23:45:00.000,6,0,60,0,0,0,60.5,Credit Card,5 Star Taxi,24,41.901206994,-87.676355989,24,41.901206994,-87.676355989,2024-10-11 23:00:00
5,f8eb56e27c4c16a5ccccccd60465e16dace863e7,75cf3a53aae5e5858361a7ca64f75d3407dc0a44d7bc42...,2024-10-11T23:45:00.000,2024-10-11T23:45:00.000,627,5.31,15.75,0,0,0,15.75,Prcard,Flash Cab,26,41.878594358,-87.730232428,28,41.874005383,-87.66351755,2024-10-11 23:00:00
6,f4f5d303d3b0d84865034080a3ba92df3f3e9558,8c302890305f37bb9d1732e606816c872a73b73f69a0ea...,2024-10-11T23:45:00.000,2024-10-12T00:00:00.000,592,2.07,45,5,0,0,50.5,Credit Card,Flash Cab,28,41.88528132,-87.6572332,32,41.870607372,-87.622172937,2024-10-11 23:00:00
7,f32a4368680bc87c304384e6c8186508dd47e927,8123ec5a70681a139fcd93fd477eec0ee05dda82987748...,2024-10-11T23:45:00.000,2024-10-11T23:45:00.000,18,0,3.25,5,0,11.75,20.5,Credit Card,Globe Taxi,19,41.927260956,-87.765501609,19,41.927260956,-87.765501609,2024-10-11 23:00:00
8,ef7d5916b592e4c3555f12a9d508c79144c02a26,a9263292f54a406cd0fb2bac808b9aaca91f770ad5e0fd...,2024-10-11T23:45:00.000,2024-10-11T23:45:00.000,180,0.5,4.75,0,0,0,4.75,Cash,Taxi Affiliation Services,32,41.880994471,-87.632746489,28,41.879255084,-87.642648998,2024-10-11 23:00:00


### taxi trips transformation functions

In [ ]:
def taxi_trips_transformations(taxi_trips:pd.DataFrame) ->pd.DataFrame:
    """Perform transformation with taxi data.

    Args:
        taxi_trips (pd.DataFrame): _description_
            The dataframe holding the daily taxi trips

    Returns:
        pd.DataFrame: _description_
            Cleaned transfromed dataframe holding the daily taxi trips.
    """
    if not isinstance(taxi_trips,pd.DataFrame):
        raise TypeError("taxi_trips is not a valid pandas Dataframe")
    
    
    
    taxi_trips.drop(["pickup_census_tract","dropoff_census_tract,dropoff_centroid_location,pickup_centroid_location"], axis=1, inplace=True)

    taxi_trips.dropna(inplace=True)

    taxi_trips.rename(columns={"pickup_community_area":"pickup_community_area_id",
    "dropoff_community_area":"dropoff_community_area_id"}, inplace=True)

    taxi_trips["datetime_for_weather"]= pd.to_datetime(taxi_trips["trip_start_timestamp"])
    taxi_trips["datetime_for_weather"]= taxi_trips["datetime_for_weather"].dt.floor("h")


    return taxi_trips

### company codes update

In [6]:
company_master = taxi_trips["company"].drop_duplicates().reset_index(drop=True)


company_master=pd.DataFrame(

{
 "company_id" : range(1,len(company_master)+1),
 "company": company_master




}
)
company_master

,company_id,company
0,1,Flash Cab
1,2,Taxicab Insurance Agency Llc
2,3,City Service
3,4,Chicago Independents
4,5,Globe Taxi
5,6,Sun Taxi
6,7,Taxi Affiliation Services
7,8,5 Star Taxi
8,9,Tac - Yellow Cab Association
9,10,Patriot Taxi Dba Peace Taxi Associat


In [7]:
new_company_data = [
{"company":"Credit_card"},
{"company": "X"},
{"company": "Y"}
]
new_company_mapping = pd.DataFrame(new_company_data)
new_company_mapping

,company
0,Credit_card
1,X
2,Y


In [8]:
company_max_id=company_master["company_id"].max()

In [9]:
new_company_mapping["company"].values

array(['Credit_card', 'X', 'Y'], dtype=object)

In [10]:
new_companies_list = []

#for company in new_company_mapping["company"].values:
#    if company not in company_master["company"].values:
#        new_companies_list.append(company)
# one line
new_companies_list = [company for company in new_company_mapping["company"].values if company not in company_master["company"].values]

In [11]:
new_companies_df = pd.DataFrame({
"company_id":range(company_max_id+1, company_max_id+ len(new_companies_list)+1),
"company": new_companies_list


})
new_companies_df

,company_id,company
0,34,Credit_card
1,35,X
2,36,Y


In [12]:
updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)
updated_company_master.tail()

,company_id,company
31,32,Metro Jet Taxi A.
32,33,Tac - Yellow Non Color
33,34,Credit_card
34,35,X
35,36,Y


In [13]:
def update_company_master(taxi_trips:pd.DataFrame,company_master:pd.DataFrame) -> pd.DataFrame:
    """Eytend the company master with companies if there are new companies

    Args:
        taxi_trips (pd.DataFrame): _description_
            Dataframe holding the daily taxi trips
        company_master (pd.DataFrame): _description_
            Dataframe holding the company master data


    Returns:
        pd.DataFrame: _description_
        The updated company master data , if new companies are in the taxi data they will be loaded to it
    """
    company_max_id=company_master["company_id"].max()
    new_companies_list = [company for company in taxi_trips["company"].values if company not in company_master["company"].values]
   
    new_companies_df = pd.DataFrame({
    "company_id":range(company_max_id+1, company_max_id+ len(new_companies_list)+1),
    "company": new_companies_list


})
    updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)

    return updated_company_master

In [14]:
taxi_trips_company_only = pd.DataFrame({

    "company_id":[1,2,3],
    "company": ["4053 - 40193 Adwar H. Nikola","X","Y"]

})
taxi_trips_company_only

,company_id,company
0,1,4053 - 40193 Adwar H. Nikola
1,2,X
2,3,Y


In [15]:
updated_company_master= update_company_master(taxi_trips_company_only, company_master)


### payment_type_master codes

In [16]:
payment_type_master = taxi_trips["payment_type"].drop_duplicates().reset_index(drop=True)


payment_type_master=pd.DataFrame(

{
 "payment_type_id" : range(1,len(payment_type_master)+1),
 "payment_type": payment_type_master
}
)
taxi_trips_payment_type_only = pd.DataFrame({

    "payment_type_id":[1,2,3],
    "payment_type": ["Credit Card","X","Y"]

})
taxi_trips_payment_type_only

,payment_type_id,payment_type
0,1,Credit Card
1,2,X
2,3,Y


In [61]:
def update_payment_type_master(taxi_trips:pd.DataFrame,payment_type_master:pd.DataFrame) -> pd.DataFrame:
    """Eytend the payment_type master with payment_type if there are new payment_types

    Args:
        taxi_trips (pd.DataFrame): _description_
            Dataframe holding the daily taxi trips
        payment_type (pd.DataFrame): _description_
            Dataframe holding the company master data


    Returns:
        pd.DataFrame: _description_
        The updated payment_type_master data , if new payment_types are in the taxi data they will be loaded to it
    """
    payment_type_max_id=payment_type_master["payment_type_id"].max()
    new_payment_types_list = [payment_type for payment_type in taxi_trips["payment_type"].values if payment_type not in payment_type_master["payment_type"].values]
   
    new_payment_type_df = pd.DataFrame({
    "payment_type_id":range(payment_type_max_id+1, payment_type_max_id+ len(new_payment_types_list)+1),
    "payment_type": new_payment_types_list


})
    updated_payment_type_master = pd.concat([payment_type_master, new_payment_type_df], ignore_index=True)

    return updated_payment_type_master

In [71]:
updated_payment_type_master= update_payment_type_master(taxi_trips_payment_type_only, payment_type_master)
updated_payment_type_master

,payment_type_id,payment_type
0,1,Credit Card
1,2,Prcard
2,3,Cash
3,4,Mobile
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,8,X
8,9,Y


### Creating a generic update master table function

In [18]:
def update_master( taxi_trips: pd.DataFrame,master: pd.DataFrame, id_column:str, value_column:str) -> pd.DataFrame:
    """Eytend the master Dataframe with new values if there are any aprameters.

    Args:
        taxi_trips (pd.DataFrame): _description_
            Dataframe holding the daily taxi trips
        master(pd.DataFrame): _description_
            Dataframe holding the master data
        id_column:str
            The id_column of the master Dataframe.
        value_column:str
            Name of the column in master_df containing the values

    Returns:
        pd.DataFrame: _description_
        The updated master data , if new values are in the taxi data they will be loaded to it
    """
    max_id=master[id_column].max()
    new_values_list = [value for value in taxi_trips[value_column].values if value not in master[value_column].values]
   
    new_values_df = pd.DataFrame({
    id_column: range(max_id+1, max_id+ len(new_values_list)+1),
    value_column: new_values_list


})
    updated_master = pd.concat([master, new_values_df], ignore_index=True)

    return updated_master

In [19]:
test_payment_type_master = update_master(taxi_trips_payment_type_only,payment_type_master,"payment_type_id","payment_type")
test_payment_type_master





,payment_type_id,payment_type
0,1,Credit Card
1,2,Mobile
2,3,Prcard
3,4,Cash
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,8,X
8,9,Y


In [20]:
test_company_master = update_master(taxi_trips_company_only,company_master,"company_id","company")
test_company_master.tail()

,company_id,company
30,31,5167 - 71969 5167 Taxi Inc
31,32,Metro Jet Taxi A.
32,33,Tac - Yellow Non Color
33,34,X
34,35,Y


### update taxi trips with the most recent company_master and payment_type master codes

In [87]:
taxi_trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tips,tolls,extras,trip_total,payment_type,company,dropoff_community_area_id,dropoff_centroid_latitude,dropoff_centroid_longitude,pickup_community_area_id,pickup_centroid_latitude,pickup_centroid_longitude,datetime_for_weather
2,fc617008a8964db298cb7d62d7c09a7331349d0b,6201cd8256e26b0ef547ca53db3aaac4d23fc7d02e4c9f...,2024-10-11T23:45:00.000,2024-10-11T23:45:00.000,6,0,60,0,0,0,60.5,Credit Card,5 Star Taxi,24,41.901206994,-87.676355989,24,41.901206994,-87.676355989,2024-10-11 23:00:00
5,f8eb56e27c4c16a5ccccccd60465e16dace863e7,75cf3a53aae5e5858361a7ca64f75d3407dc0a44d7bc42...,2024-10-11T23:45:00.000,2024-10-11T23:45:00.000,627,5.31,15.75,0,0,0,15.75,Prcard,Flash Cab,26,41.878594358,-87.730232428,28,41.874005383,-87.66351755,2024-10-11 23:00:00
6,f4f5d303d3b0d84865034080a3ba92df3f3e9558,8c302890305f37bb9d1732e606816c872a73b73f69a0ea...,2024-10-11T23:45:00.000,2024-10-12T00:00:00.000,592,2.07,45,5,0,0,50.5,Credit Card,Flash Cab,28,41.88528132,-87.6572332,32,41.870607372,-87.622172937,2024-10-11 23:00:00
7,f32a4368680bc87c304384e6c8186508dd47e927,8123ec5a70681a139fcd93fd477eec0ee05dda82987748...,2024-10-11T23:45:00.000,2024-10-11T23:45:00.000,18,0,3.25,5,0,11.75,20.5,Credit Card,Globe Taxi,19,41.927260956,-87.765501609,19,41.927260956,-87.765501609,2024-10-11 23:00:00
8,ef7d5916b592e4c3555f12a9d508c79144c02a26,a9263292f54a406cd0fb2bac808b9aaca91f770ad5e0fd...,2024-10-11T23:45:00.000,2024-10-11T23:45:00.000,180,0.5,4.75,0,0,0,4.75,Cash,Taxi Affiliation Services,32,41.880994471,-87.632746489,28,41.879255084,-87.642648998,2024-10-11 23:00:00


In [21]:
def update_taxi_trips_with_master_data(taxi_trips:pd.DataFrame,payment_type_master:pd.DataFrame,company_master: pd.DataFrame):
    """Update the taxi_trips dataframe with the company_master and payment_master ids, and delete the string columns.

    Args:
        taxi_trips (pd.DataFrame): _description_
            the dataframe with the daily taxi trips
        payment_type_master (pd.DataFrame): _description_
            The payment master table
        company_master (pd.DataFrame): _description_
            The company master table

    Returns:
        _type_: _description_
    """

    taxi_trips_id = taxi_trips.merge(payment_type_master,on="payment_type")
    taxi_trips_id = taxi_trips_id.merge(company_master,on="company")
    taxi_trips_id.drop(["payment_type","company"],axis=1, inplace=True)

    return taxi_trips_id

In [22]:
taxi_trips_id = update_taxi_trips_with_master_data(taxi_trips,payment_type_master,company_master)
taxi_trips_id

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
0,be2e655975f3b0408a96cfeb74591c49fe460f66,5d7801992a88d2d1340e703e6ff8efd44302443c3e4058...,2024-10-21T23:45:00.000,2024-10-22T00:15:00.000,1385,18.4,76,8,45.25,7.61,0,5,58.36,41.980264315,-87.913624596,41.899602111,-87.633308037,2024-10-21 23:00:00,1,1
1,2105c51decaebdbbd3aa29a27d50403900284d66,9f87b11be025b5dcc55e9472b8c2158664c97d193cf15d...,2024-10-21T23:45:00.000,2024-10-22T00:00:00.000,1074,11.18,56,33,28.75,6.85,0,5,41.1,41.79259236,-87.769615453,41.857183858,-87.620334624,2024-10-21 23:00:00,2,2
2,2684268f8cd486588d57b037dce95a7d4c71af63,02ef8f01232b1b1828f4e5e1b8e8a85cd71b67c449afaf...,2024-10-21T23:45:00.000,2024-10-22T00:15:00.000,1780,13.61,44,8,34.25,0,0,0,34.25,41.740205756,-87.615969523,41.899602111,-87.633308037,2024-10-21 23:00:00,3,3
3,274aea2de3074f4af1b0165a321f03eb31745ab1,d6d5194c8f712167fb1c63ef4409ce18414e022c65b7e6...,2024-10-21T23:45:00.000,2024-10-22T00:15:00.000,1432,13.8,56,8,36,4,0,5,45.5,41.79259236,-87.769615453,41.899602111,-87.633308037,2024-10-21 23:00:00,1,4
4,2878929079799757dd8e6ed0d3b18422ae39ba97,8aa56dbaf30cae892a59e732f54d5eb6694d054fd459e3...,2024-10-21T23:45:00.000,2024-10-22T00:00:00.000,1424,15.4,76,7,38.25,8.75,0,5,52.5,41.980264315,-87.913624596,41.922686284,-87.649488729,2024-10-21 23:00:00,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18561,20d8ebe48ce86fa5ca80a6b88b59d4fb569f20aa,c018902b2cca3d3d34af3de6d486f3a7dd7af724d85ce5...,2024-10-21T00:00:00.000,2024-10-21T00:30:00.000,1152,14.16,76,7,35.75,0,0,4,39.75,41.980264315,-87.913624596,41.922686284,-87.649488729,2024-10-21 00:00:00,4,2
18562,16f47fd716a372564ac89ddf3d61c44952bc9bfc,78acc8b3d6416687af307c51877d66d84d2f3ba1e04a16...,2024-10-21T00:00:00.000,2024-10-21T00:15:00.000,540,0,28,24,22,0,0,0,22,41.874005383,-87.66351755,41.901206994,-87.676355989,2024-10-21 00:00:00,1,13
18563,15ab89eff8dd982fdb8a623bf05450266c1bc1cb,ddc0a237498a82a0803761188d0ea048623b5d91f29ad2...,2024-10-21T00:00:00.000,2024-10-21T00:15:00.000,963,3.45,8,7,10.51,5,0,0,15.51,41.899602111,-87.633308037,41.922686284,-87.649488729,2024-10-21 00:00:00,2,1
18564,10f5b63a1949affe515a73c0e8e4a0b131debc67,70d9077f1ddbc6c93643ef4dbf4a3110b1826f68a69801...,2024-10-21T00:00:00.000,2024-10-21T00:15:00.000,1020,13.5,76,22,34,7.7,0,4,45.7,41.980264315,-87.913624596,41.92276062,-87.699155343,2024-10-21 00:00:00,1,7


#### weather transformation function

In [96]:
def tansform_weather_data (weather_data: json) ->pd.DataFrame:
    """make daily transformation on the daily weather API response

    Args:
        weather_data (json): _description_
        Daily weather data from the Open Meteo API
    Returns:
        pd.DataFrame: _description_
            A Dataframe representation of the weather data
    """
    weather_data_filtered = {
        "datetime":weather_data ["hourly"]["time"],
        "temperature":weather_data ["hourly"]["temperature_2m"],
        "wind_speed":weather_data ["hourly"]["wind_speed_10m"],
        "rain":weather_data ["hourly"]["rain"],
        "precipitation":weather_data ["hourly"]["precipitation"],
}

    weather_df=pd.DataFrame(weather_data_filtered)
    weather_df["datetime"] = pd.to_datetime(weather_df["datetime"])

    return weather_df

In [98]:
current_datetime = datetime.now() - relativedelta(months=2)
formatted_datetime= current_datetime.strftime("%Y-%m-%d")

url= "https://archive-api.open-meteo.com/v1/era5"



params={
    "latitude" :41.85,
    "longitude" :-87.68,
    "start_date" :formatted_datetime,
    "end_date" : formatted_datetime,
    "hourly" : "temperature_2m,wind_speed_10m,precipitation,rain"   
     }
response= requests.get(url,params=params)
weather_data = response.json()

weather_data_df = tansform_weather_data(weather_data)